In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [5]:
train_path = '/content/data/train'
test_path = '/content/data/test'

In [3]:
# !rmdir '/content/data/train/.ipynb_checkpoints'

In [6]:
# Define the model
model = Sequential()

# Input layer and first convolutional layer
model.add(Conv2D(
    filters=32,
    kernel_size=(3, 3),
    activation='relu',
    input_shape=(200, 200, 3)
))

# Max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten layer to convert 2D data to 1D
model.add(Flatten())

# Fully connected layer with 64 neurons
model.add(Dense(units=64, activation='relu'))

# Output layer for binary classification
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
optimizer = SGD(learning_rate=0.002, momentum=0.8)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 198, 198, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 99, 99, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 313632)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │      20,072,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
train_gen = ImageDataGenerator(rescale=1./255)

train_ds = train_gen.flow_from_directory(
    train_path,
    target_size=(200, 200),
    batch_size=20,
    shuffle = True,
    class_mode = "binary"
)

test_gen = ImageDataGenerator(rescale=1./255)

test_ds = test_gen.flow_from_directory(
    test_path,
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode = "binary"
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [8]:
steps_per_epoch = train_ds.n//train_ds.batch_size
validation_steps = test_ds.n//test_ds.batch_size

10

In [16]:
history = model.fit(
    train_ds,
    epochs=10,
    steps_per_epoch = steps_per_epoch,
    validation_data=test_ds,
    validation_steps = validation_steps
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 136ms/step - accuracy: 0.8182 - loss: 0.4247 - val_accuracy: 0.6550 - val_loss: 0.6102
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0192
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 113ms/step - accuracy: 0.8165 - loss: 0.3999 - val_accuracy: 0.6600 - val_loss: 0.5886
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 317us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.1212
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 114ms/step - accuracy: 0.8153 - loss: 0.3729 - val_accuracy: 0.6800 - val_loss: 0.6558
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 323us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.2039
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 133ms/step - accuracy: 0.8257 - loss: 0.3649 - val_accuracy: 0.6850 - val_loss: 0.6581
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss:

In [17]:
np.median(history.history['accuracy'])

0.4037500023841858

In [18]:
np.std(history.history['loss'])

0.19464583959561965

In [19]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale the pixel values to [0, 1]
    rotation_range=50,  # Random rotations in the range of 50 degrees
    width_shift_range=0.1,  # Random horizontal shift of 10%
    height_shift_range=0.1,  # Random vertical shift of 10%
    zoom_range=0.1,  # Random zoom of up to 10%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill missing pixels with the nearest pixel value
)

train_ds = train_gen.flow_from_directory(
    train_path,
    target_size=(200, 200),
    batch_size=20,
    shuffle = True,
    class_mode = "binary"
)

test_gen = ImageDataGenerator(rescale=1./255)

test_ds = test_gen.flow_from_directory(
    test_path,
    target_size=(200, 200),
    batch_size=20,
    shuffle=True,
    class_mode = "binary"
)

Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [20]:
history = model.fit(
    train_ds,
    epochs=20,
    initial_epoch=10,
    steps_per_epoch = steps_per_epoch,
    validation_data=test_ds,
    validation_steps = validation_steps
)

Epoch 11/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 117ms/step - accuracy: 0.8898 - loss: 0.3068 - val_accuracy: 0.7300 - val_loss: 0.6496
Epoch 12/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0166
Epoch 13/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 139ms/step - accuracy: 0.9186 - loss: 0.2551 - val_accuracy: 0.6700 - val_loss: 0.6027
Epoch 14/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.5210
Epoch 15/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 127ms/step - accuracy: 0.9337 - loss: 0.2279 - val_accuracy: 0.6950 - val_loss: 0.6005
Epoch 16/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 393us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0463
Epoch 17/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 116ms/step - accuracy: 0.9318 - loss: 0.2283 - val_accuracy: 0.7250 - val_loss: 0.6402
Epoch 18/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss: 0.00

In [21]:
print(np.mean(history.history['val_loss']))
print(np.mean(history.history['val_accuracy'][5:]))

0.39506479394622146
0.8879999995231629
